In [1]:
#imports
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import acquire as a
import os
import model as m
from sklearn.model_selection import train_test_split
from scipy import stats
from sklearn.cluster import KMeans


In [2]:
df = a.wrangle_wine()
df.head()

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,quality,type
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,red
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5,red
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5,red
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6,red
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,red


In [3]:
train, val, test = a.split_data(df)
train.shape, val.shape, test.shape

((3637, 13), (1560, 13), (1300, 13))

In [4]:
train.head()

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,quality,type
4085,6.7,0.24,0.32,10.3,0.079,37.0,122.0,0.99662,3.02,0.45,8.8,5,white
1402,7.6,0.30,0.42,2.0,0.052,6.0,24.0,0.99630,3.44,0.82,11.9,6,red
2137,6.1,0.24,0.30,1.5,0.045,22.0,61.0,0.99200,3.31,0.54,10.4,5,white
1482,7.2,0.58,0.54,2.1,0.114,3.0,9.0,0.99719,3.33,0.57,10.3,4,red
3857,7.6,0.48,0.28,10.4,0.049,57.0,205.0,0.99748,3.24,0.45,9.3,5,white


In [5]:
to_scale = a.get_continuous_feats(df)

train, val, test = a.scale_data(train, val, test, to_scale)
train.head()

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,quality,type
4085,0.239316,0.128,0.32,0.148773,0.116279,0.276923,0.377850,0.183343,0.241379,0.129213,0.129032,5,white
1402,0.316239,0.176,0.42,0.021472,0.071429,0.038462,0.058632,0.177174,0.603448,0.337079,0.629032,6,red
2137,0.188034,0.128,0.30,0.013804,0.059801,0.161538,0.179153,0.094274,0.491379,0.179775,0.387097,5,white
1482,0.282051,0.400,0.54,0.023006,0.174419,0.015385,0.009772,0.194332,0.508621,0.196629,0.370968,4,red
3857,0.316239,0.320,0.28,0.150307,0.066445,0.430769,0.648208,0.199923,0.431034,0.129213,0.209677,5,white


In [6]:
train, val, test = a.dummies(train, val, test)

In [7]:
X_train, y_train = a.xy_split(train)
X_val, y_val = a.xy_split(val)

In [8]:
X_train.head()

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,type_red,type_white
4085,0.239316,0.128,0.32,0.148773,0.116279,0.276923,0.377850,0.183343,0.241379,0.129213,0.129032,0,1
1402,0.316239,0.176,0.42,0.021472,0.071429,0.038462,0.058632,0.177174,0.603448,0.337079,0.629032,1,0
2137,0.188034,0.128,0.30,0.013804,0.059801,0.161538,0.179153,0.094274,0.491379,0.179775,0.387097,0,1
1482,0.282051,0.400,0.54,0.023006,0.174419,0.015385,0.009772,0.194332,0.508621,0.196629,0.370968,1,0
3857,0.316239,0.320,0.28,0.150307,0.066445,0.430769,0.648208,0.199923,0.431034,0.129213,0.209677,0,1


In [9]:
y_train.mean(), y_train.median()

(5.816057189991751, 6.0)

In [10]:
baselines = pd.DataFrame({'y_actual': y_train,
                         'y_mean': y_train.mean(),
                         'y_median': y_train.median()})

baselines.head()

,y_actual,y_mean,y_median
4085,5,5.816057,6.0
1402,6,5.816057,6.0
2137,5,5.816057,6.0
1482,4,5.816057,6.0
3857,5,5.816057,6.0


In [11]:
m.eval_model(baselines.y_actual, baselines.y_mean)

0.8647850872965321

In [12]:
m.eval_model(baselines.y_actual, baselines.y_median)

0.884131327668125

In [13]:
lm = LinearRegression()

m.train_model(lm, X_train, y_train, X_val, y_val)

NameError: name 'LinearRegression' is not defined

In [ ]:
X= train[['alcohol', 'density']]

kmeans = KMeans(n_clusters=3)
kmeans.fit(X)

kmeans.predict(X)


In [ ]:
strain['cluster'] = kmeans.predict(X)

In [ ]:
kmeans.cluster_centers_

In [ ]:
centroids = pd.DataFrame(kmeans.cluster_centers_, columns=X.columns)
centroids